# Merge
Duncan Callaway

This notebook gives an introduction to using Pandas' `merge` method.  

In [1]:
import pandas as pd
import numpy as np

## Row and column labels
The columns are identified with a list of values.  Let's look at the fruit data set again:

In [2]:
fruit_info_df = pd.read_csv('fruit_info.csv', index_col= False)
fruit_info_df

,fruit,color,weight
0,apple,red,120
1,banana,yellow,150
2,orange,orange,250
3,raspberry,pink,15


### Q: How do I print out just the columns?

In [3]:
fruit_info_df.columns

Index(['fruit', 'color', 'weight'], dtype='object')

### Q: And the rows?

The rows are similarly labeled:

In [4]:
fruit_info_df.index

RangeIndex(start=0, stop=4, step=1)

## Merging
Lets make another data frame and tack it on to the first

In [5]:
price_df = pd.DataFrame({'price':[0.5, 0.65, 1, 0.15],
                        'frut':['apple', 'banana', 'orange', 'rasberry']})
price_df

,price,frut
0,0.50,apple
1,0.65,banana
2,1.00,orange
3,0.15,rasberry


Now let's blindly merge:

In [6]:
pd.merge(price_df,fruit_info_df)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

What went wrong?

First, we didn't spell fruit correctly.  Two ways to fix.  First, specify the columns directly:

In [7]:
pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit')

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120
1,0.65,banana,banana,yellow,150
2,1.00,orange,orange,orange,250


Second, fix the spelling and *don't* tell pandas.  In this case pandas works to figure out what's in common.

In [8]:
price_df.columns[0]='fruit'

TypeError: Index does not support mutable operations

Bummer!  Can't mutate index values.  What to do?

In [9]:
col_list = list(price_df.columns)
col_list

['price', 'frut']

In [10]:
col_list[1] = 'fruit'

In [11]:
price_df.columns = col_list
price_df

,price,fruit
0,0.50,apple
1,0.65,banana
2,1.00,orange
3,0.15,rasberry


In [12]:
pd.merge(fruit_info_df,price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00


Note we can use different syntax:

In [13]:
fruit_info_df.merge(price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00


### Q: Now we're still missing raspberries -- why?

Again, spelling error in the new frame.  Let's fix:

In [14]:
price_df.loc[3,'fruit'] = 'raspberry'

Note we could change individual entries in the data frame itself.  They are mutable.

In [15]:
fruit_info_df.merge(price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00
3,raspberry,pink,15,0.15


Another few things to takeaway from this
1. Inner merge -- the default -- is brutal.  That is, it'll drop data without telling you.  Other ways to merge are a little less draconian.
2. It's important to review your results.  How many rows do you expect?  How many do you actually get?  Did something important get chucked out?  The ensuing solutions are the non-glamorous tasks of data cleaning.

In [16]:
merged_df = fruit_info_df.merge(price_df)
merged_df

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00
3,raspberry,pink,15,0.15


## Merge Types: Inner, Outer, Left, Right

Let's load up the dataframe again and experiment with different merge types:

In [17]:
fruit_info_df = pd.read_csv('fruit_info.csv', index_col= False)
price_df = pd.DataFrame({'price':[0.5, 0.65, 1, 0.15],
                        'frut':['apple', 'banana', 'orange', 'rasberry']})

In [18]:
merged_df_inner = pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit', how = 'inner')
merged_df_inner

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120
1,0.65,banana,banana,yellow,150
2,1.00,orange,orange,orange,250


That's what we got above.  `pd.merge` gives an inner join by default.

### Q: Try doing an outer, left, and right merge. 

In [19]:
merged_df_outer = pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit', how = 'outer')
merged_df_outer

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120.0
1,0.65,banana,banana,yellow,150.0
2,1.00,orange,orange,orange,250.0
3,0.15,rasberry,NaN,NaN,NaN
4,NaN,NaN,raspberry,pink,15.0


You can see we kept *every* row from both dataframes, and populated with NaNs where keys don't match.

Let's try left and right:

In [20]:
merged_df_left = pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit', how = 'left')
merged_df_left

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120.0
1,0.65,banana,banana,yellow,150.0
2,1.00,orange,orange,orange,250.0
3,0.15,rasberry,NaN,NaN,NaN


In [21]:
merged_df_right = pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit', how = 'right')
merged_df_right

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120
1,0.65,banana,banana,yellow,150
2,1.00,orange,orange,orange,250
3,NaN,NaN,raspberry,pink,15


We can streamline by replacing the index number with the fruit column.  

### Q: in the following, what's the `inplace` command for?  

In [22]:
merged_df.set_index('fruit', inplace = True)
merged_df

,color,weight,price
fruit,,,
apple,red,120,0.50
banana,yellow,150,0.65
orange,orange,250,1.00
raspberry,pink,15,0.15


### A: It means the re-defined dataframe is assigned to the original name.  
This is advantageous in memory constrained situations.  

## Multilevel indexing 

We can also assign "multilevel" column or row names, like so:

In [23]:
levels = [('categorical', 'color'),('quantitative', 'weight'),('quantitative','price')]
levels

[('categorical', 'color'),
 ('quantitative', 'weight'),
 ('quantitative', 'price')]

Note the  use of tuples (sets of values in parentheses) in setting up multiindex.  This will come again later.  

In [24]:
merged_df.columns = pd.MultiIndex.from_tuples(levels)
merged_df

categorical quantitative      
                color       weight price
fruit                                   
apple             red          120  0.50
banana         yellow          150  0.65
orange         orange          250  1.00
raspberry        pink           15  0.15

Now we have categories and subcategories of columns:

In [25]:
merged_df['quantitative']

,weight,price
fruit,,
apple,120,0.50
banana,150,0.65
orange,250,1.00
raspberry,15,0.15


### Q: How can we get data from an individual column?
Aim to get the `weight` column:

In [26]:
merged_df[('quantitative','weight')]

fruit
apple        120
banana       150
orange       250
raspberry     15
Name: (quantitative, weight), dtype: int64

## Advanced multilevel (did not do in lecture) 
Note, we can also drop and add things.  With multilevel indexing things get a little tricky.  

First, we can drop everything from the top level:

In [30]:
merged_test_df = merged_df.drop(columns=[('quantitative',)], axis = 1)
merged_test_df

,categorical
,color
fruit,
apple,red
banana,yellow
orange,orange
raspberry,pink


Note that I put the column identifier inside the parens, like a tuple, but it's not essential there.

However if we want to drop only a column from the second level, we get an error without the tuple syntax:

In [31]:
merged_test_df = merged_df.drop(columns=[('quantitative','price')], axis = 1)
merged_test_df

,categorical,quantitative
,color,weight
fruit,,
apple,red,120
banana,yellow,150
orange,orange,250
raspberry,pink,15


We can also drop rows: 

In [32]:
merged_df.drop(index=[('apple')], axis = 0, inplace = True)
merged_df

categorical quantitative      
                color       weight price
fruit                                   
banana         yellow          150  0.65
orange         orange          250  1.00
raspberry        pink           15  0.15

Note indexing multilevels with `.loc` gets a little tricky.  The thing to keep in mind is that you're working with tuples in each index location:

In [33]:
merged_df.loc['banana', ('quantitative', 'price')]

np.float64(0.65)

If you leave an entry of the tuple empty you get all values.  

In [34]:
merged_df.loc['banana', ('quantitative', )]

weight    150.00
price       0.65
Name: banana, dtype: float64

You can also loop through the columns of the multilevel data frame like this: 

In [35]:
for i, j in merged_df:
    print(merged_df.loc['banana', (i, j)])

yellow
150
0.65


Some added thoughts:
1. Multilevel indexing works for columns and index
2. It can be a powerful way to summarize your data and quickly reference subsets of it.
4. However it can also be a colossal pain in the rear -- indexing with multilevel is often very hard to parse and debug.